In [1]:
# Imports with all of my tools
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import cv2
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import os
import random
from sympy import mod_inverse
from PIL import Image as im

%matplotlib inline


def get_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    print(image.shape)
    return image

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
path = r"C:\Users\soule\OneDrive\Pictures\addie.jpg"
l = get_image(path)
img = im.open(path)
colors = img.getpixel((0, 1))
print(colors)
rows, cols = l.shape
# print(l)
data = im.fromarray(l.astype(np.uint8))
data.show()

(450, 600)
(30, 31, 35)


<IPython.core.display.Javascript object>

In [4]:
def Generate_Point(a, b, p):
    x = random.randint(0, p)
    print(x)
    y = np.arange(0, p)
    possibilites = []
    for i in range(len(y)):
        yval = int((x ** 3) % p)
        yval = int((yval + (a * x)) % p)
        yval = int((yval + b) % p)
        # yval = ((x ** 3) + (a * x) + b) % p
        # print(yval)
        # print(x[i],y[j],y[j]**2 % p, yval)
        # print(yval)
        testy = int((y[i] ** 2) % p)
        if testy == yval:
            possibilites.append(y[i])
            print(possibilites)
    if len(possibilites) == 0:
        return Generate_Point(a, b, p)
    else:
        yindex = random.randint(0, len(possibilites) - 1)
        return x, possibilites[yindex]


def point_addition(x1, y1, x2, y2, p, a):
    z = 0
    if x1 == z and y1 == z:
        return x2, y2
    if x2 == z and y2 == z:
        return x1, y1
    if x1 == x2:
        if x1 == x2 and y1 == -y2 % p:
            return 0, 0
        else:
            den = int(mod_inverse(2 * y1, p) % p)
            m = int((3 * (x1 ** 2)) % p)
            m = int((m + a) % p)
            m = int((m * den) % p)
            m = int((((3 * (x1 ** 2)) % p + a) * (den)) % p)
            # m = (((3*(x1**2))%p + a) * (2*))
            # m = ((3*pow(x1, 2, p) + a)*pow(2*y1, p-2, p)) % p
            # print(m)
    else:
        den = int(mod_inverse(x1 - x2, p) % p)
        m = int((y1 - y2) % p)
        m = int((m * den) % p)
        # m = int(((y1 - y2)*den)%p)
        # print(m)
    x = int((m ** 2) % p)
    x = int((x - x1 - x2) % p)
    # x = int(((m**2)%p - x1 - x2) % p)
    # print(x)
    y = int((x1 - x) % p)
    y = int((y * m) % p)
    y = int((y - y1) % p)
    # y = int((m*(x1 - x)%p - y1) % p)
    # print(y)
    return int(x), int(y)


def point_subtraction(x1, y1, x2, y2, p, a):
    y2 = int(-(y2))
    y2 = int(y2 % p)
    return point_addition(x1, y1, x2, y2, p, a)


def point_multiplication(x, y, k, p, a):
    xnew, ynew = int(x), int(y)
    for i in range(k - 1):
        xnew, ynew = point_addition(x, y, xnew, ynew, p, a)
        # print(xnew,ynew)
    return int(xnew), int(ynew)

<IPython.core.display.Javascript object>

In [5]:
arr = []
for i in range(rows):
    for j in range(cols):
        arr.append(l[i][j])
print(arr[0], arr[1], arr[2])
print(l)

30 31 33
[[ 30  31  33 ...  71  79  95]
 [ 31  32  34 ...  76  74  99]
 [ 31  32  34 ... 105  87 113]
 ...
 [ 90  88  85 ... 116  98  91]
 [ 87  89  78 ... 111 120 101]
 [ 66  63  48 ... 110 129 101]]


<IPython.core.display.Javascript object>

In [6]:
totalPixs = rows * cols
print(totalPixs)

270000


<IPython.core.display.Javascript object>

In [7]:
imagePointsArray = np.reshape(arr, (totalPixs // 2, 2), order="C")
print(imagePointsArray)

[[ 30  31]
 [ 33  36]
 [ 39  41]
 ...
 [ 85  92]
 [ 97 110]
 [129 101]]


<IPython.core.display.Javascript object>

In [8]:
p = 16769023 #1046527  # 16127
a = -5000
b = 76193

<IPython.core.display.Javascript object>

In [9]:
# armins private keys
private_A = 712
point_A = Generate_Point(a, b, p)
print(point_A)

823874
1037795
[502249]
[502249, 580335]
(1037795, 502249)


<IPython.core.display.Javascript object>

In [10]:
# armins public key
public_A = point_multiplication(point_A[0], point_A[1], private_A, p, a)
print(public_A)

(387108, 87436)


<IPython.core.display.Javascript object>

In [11]:
# Erins key is 92
e_key = 92

<IPython.core.display.Javascript object>

In [12]:
coordinates, variables = imagePointsArray.shape
c2 = []
print(coordinates)
r = coordinates
c1 = point_multiplication(point_A[0], point_A[1], e_key, p, a)
added_to_M = point_multiplication(public_A[0], public_A[1], e_key, p, a)
for i in range(coordinates):
    c2_operation = point_addition(
        imagePointsArray[i][0],
        imagePointsArray[i][1],
        added_to_M[0],
        added_to_M[1],
        p,
        a,
    )
    c2.append(c2_operation[0])
    c2.append(c2_operation[1])
print(len(c2))

135000
270000


<IPython.core.display.Javascript object>

final_encrypted_array = []
for i in range(len(encrypted_array)):
    scaled_value = encrypted_array[i] % 256
    divisor = encrypted_array[i] // 256
    final_encrypted_array.append(scaled_value)
    final_encrypted_array.append(divisor)

In [13]:
encryption_arr = np.array(c2, dtype=int)
encryption_arr = np.reshape(encryption_arr, (rows, cols), order="C")
print(encryption_arr)
e2 = encryption_arr % 256
print(e2)

[[ 403024  303319  585944 ...  469593   31161  116384]
 [ 532564  804361 1021802 ...  853991  258189  648316]
 [ 532564  804361 1021802 ...  987240  331007 1020112]
 ...
 [ 493277  454838  582381 ...  267853  282256  209762]
 [  37827  577133  312164 ...  217268  241277  293404]
 [ 971059  759249  415011 ...  947933  301704  409426]]
[[ 80 215 216 ...  89 185 160]
 [ 84   9 106 ... 231 141 124]
 [ 84   9 106 ... 104 255 208]
 ...
 [221 182 237 ...  77 144  98]
 [195 109 100 ... 180 125  28]
 [ 51 209  35 ... 221 136  82]]


<IPython.core.display.Javascript object>

In [14]:
print(459362 % 256)

98


<IPython.core.display.Javascript object>

In [15]:
e_data = im.fromarray(e2.astype(np.uint8))
e_data.show()

<IPython.core.display.Javascript object>

# Decryption
data = encryption_arr
x, y = data.shape
print(x, y)
print(data.shape)
# print(data)

In [16]:
arr = []
for i in range(rows):
    for j in range(cols):
        arr.append(encryption_arr[i][j])
print(len(arr))
print(arr[0], arr[1], arr[2], arr[3])
# print(arr)

270000
403024 303319 585944 65361


<IPython.core.display.Javascript object>

apply_divisor = np.reshape(encryption_arr, (len(arr) // 2, 2), order="C")
applied_data = []
for i in range(len(arr) // 2):
    newValue = apply_divisor[i][1] * 256 + apply_divisor[i][0]
    applied_data.append(newValue)
print(len(applied_data))

x = x // 2

In [17]:
# newTotal = x * y
EmPointsArray = np.reshape(arr, (totalPixs // 2, 2), order="C")
print(EmPointsArray)

[[403024 303319]
 [585944  65361]
 [ 39609 738748]
 ...
 [627935 648487]
 [ 94730 947933]
 [301704 409426]]


<IPython.core.display.Javascript object>

In [18]:
coordinates, variables = EmPointsArray.shape
subtract_from_c2 = point_multiplication(c1[0],c1[1],private_A,p,a)
decrypted_array = []
print(coordinates)
r=coordinates
for i in range(coordinates):
    #C_E - (alpha * (G+B_1) + B_A)
    DP = point_subtraction(
        EmPointsArray[i][0],
        EmPointsArray[i][1],
        subtract_from_c2[0],
        subtract_from_c2[1],
        p,
        a
    )
    decrypted_array.append(DP[0])
    decrypted_array.append(DP[1])


135000


<IPython.core.display.Javascript object>

In [19]:
decryption_arr = np.array(decrypted_array, dtype=int)
decryption_arr = np.reshape(decryption_arr, (rows, cols), order="C")
decryption_arr

array([[ 30,  31,  33, ...,  71,  79,  95],
       [ 31,  32,  34, ...,  76,  74,  99],
       [ 31,  32,  34, ..., 105,  87, 113],
       ...,
       [ 90,  88,  85, ..., 116,  98,  91],
       [ 87,  89,  78, ..., 111, 120, 101],
       [ 66,  63,  48, ..., 110, 129, 101]])

<IPython.core.display.Javascript object>

In [20]:
d_data = im.fromarray(decryption_arr.astype(np.uint8))
d_data.show()

<IPython.core.display.Javascript object>